*Bouncer

In [7]:
from slackclient import SlackClient
import psycopg2 as pg
import pandas.io.sql as psql
#from datetime import datetime
import datetime
import sys
import json
import requests

print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)
conn_str = "host={0} port={1} dbname={2} user={3} password={4}" \
.format("localhost", 5432, "kanjidb", "postgres", "w0lfpack")

try:
    conn = pg.connect(conn_str)
    print("Welcome to Jupyter Notebook.  You are connected to the Kanji  database!")
except pg.OperationalError:
    print("You are not connected to the database.")

Python version
3.7.2 (default, Dec 29 2018, 06:19:36) 
[GCC 7.3.0]
Version info.
sys.version_info(major=3, minor=7, micro=2, releaselevel='final', serial=0)
Welcome to Jupyter Notebook.  You are connected to the Kanji  database!


In [8]:
_STATUS_ONLINE = 10001
_REPORT_INTERVAL_MINS = 15
_ALERT_THRESHOLD_COUNT = 10
_DEPLOY_STATE_ACTIVE = 10001
#_MESH_NETWORK_ID = sys.argv[1]
_MESH_NETWORK_ID = 10000
_BOUNCE_THRESHOLD = 5
_GATEWAY_CORE_TYPE = 10003

_PARTICLE_TOKEN = '8fbca1b14ed0a064acaab9fc1dad40fdf124186a'

_CHANNEL_NAME = "meshio"

_BANNER_ALERT_MESSAGE = "Node Alert"
_BANNER_OK_MESSAGE = "Sensor Network"
_SLACK_EMOJI = ':warning:'

_BANNER_BOUNCE_MESSAGE = "Kanji Bouncer"
_SLACK_EMOJI = ':bomb:'

In [9]:
messagetemplate = "[\
   {\"type\": \"section\", \
		\"text\": { \
			\"type\": \"mrkdwn\", \
			\"text\": \"*<fakeLink.toUserProfiles.com|Iris / Zelda 1-1>*\\nTuesday, January 21 4:00-4:30pm\\nBuilding 2 - Havarti Cheese (3)\\n2 guests\" \
		}, \
		\"accessory\": { \
			\"type\": \"image\", \
			\"image_url\": \"https://api.slack.com/img/blocks/bkb_template_images/notifications.png\", \
			\"alt_text\": \"calendar thumbnail\" \
		} \
	}, \
    { \
		\"type\": \"divider\" \
	} ]"

nodetemplate = "{\"type\": \"context\", \
		\"elements\": [ \
			{ \
				\"type\": \"image\", \
				\"image_url\": \"https://www.dropbox.com/s/o2gjgfsph2cxgds/thermometer.jpg?raw=1\", \
				\"alt_text\": \"notifications warning icon\" \
			}, \
			{ \
				\"type\": \"mrkdwn\", \
				\"text\": \"*Conflicts with Team Huddle: 4:15-4:30pm*\" \
			} \
		] \
	}"


locationimageurl = 'https://www.dropbox.com/s/hunqb8wd55qdmqt/gear-hammer-wrench.png?raw=1'

In [10]:
now = datetime.datetime.now()
blockmessage = json.loads(messagetemplate)
nodemessage  = json.loads(nodetemplate)

timestart = now - datetime.timedelta(hours=1, minutes=_REPORT_INTERVAL_MINS)
querytime = timestart.strftime("%Y/%m/%d %H:%M:%S")
print("report period {0} - {1}".format(timestart, now))

#We are looking for evidence that a gateway should be recycled
#Only scan for ACTIVE devices, we don't want to bounce gateway for nodes in MAINTENANCE mode
query = "SELECT name, idnode, coreid, coretype_id, location_id FROM kanji_node WHERE meshnetwork_id={0} AND deploystate_id={1}".format(_MESH_NETWORK_ID, _DEPLOY_STATE_ACTIVE)

print(query)

print("Today is {0}".format(now))
cur = conn.cursor()
cur.execute(query)
rows = cur.fetchall()
ndown = 0
if len(rows) > 0:    
    print("The query returned the following:") 
    for row in rows:
        nodename = row[0]
        nodeid = row[1]
        coreid = row[2]
        coretype_id = row[3]
        if coretype_id==_GATEWAY_CORE_TYPE:
            gatewayid = coreid
            gatewayidnode = nodeid
            gatewayname = nodename 
            gatewaylocationid = row[4]
        q2 = "SELECT node_id, COUNT(*) FROM kanji_pinglog WHERE node_id={0} AND pingstate_id={1} AND timestamp>= NOW() - INTERVAL '{2} minutes'  GROUP BY node_id".format(nodeid, _STATUS_ONLINE, _REPORT_INTERVAL_MINS)
        #print(q2)
        cur.execute(q2)
        count = cur.fetchone()
        if count:
            print("{0} was down {1} times in the period.".format(nodename, count[1]))
            ndown += int(count[1])
        else:
            print("{0} was up for the entire period.".format(nodename))
print("There were a total of {0} downed nodes in the period.".format(ndown))
if ndown >= _BOUNCE_THRESHOLD:
    print("We'll bounce the gateway (coreid={0}) for this mesh network".format(gatewayid))
    BOUNCE_URL = "https://api.particle.io/v1/devices/{0}/bounce".format(gatewayid)
    print(BOUNCE_URL)
    PARAMS = {'access_token' : _PARTICLE_TOKEN, 'arg' : "_RESET_SECRET"}
    print(PARAMS)
    try: 
      resp = requests.post(BOUNCE_URL, data = PARAMS, timeout=(15, 30))
      response = resp.json()
      print(response)
    except (requests.exceptions.ReadTimeout):
      print("The bounce request timed out as expected!")
    query = "SELECT location.description, customer.slacktoken \
    FROM kanji_location location \
    JOIN kanji_customer customer ON location.customer_id=customer.idcustomer \
    WHERE idlocation={0}".format(gatewaylocationid)
    cur = conn.cursor()
    cur.execute(query)
    row = cur.fetchone()
    location = row[0]
    _SLACK_TOKEN = row[1]
    
    # start key replacement
    # the troubling image
    print(blockmessage[0]["accessory"]["image_url"])
    blockmessage[0]["accessory"]["image_url"] = locationimageurl

    # *Porter Farms - Building 1  / 02:15 PM Monday, March 18, 2019 
    # The current building temperature is 69.2° F
    blockmessage[0]["text"]["text"] = "*{0}* \
    \nAt {1} \
    \n*Bounce-O-Matic is restarting {2} gateway. (GID={3})*".format(location, now.strftime("%-I:%M %p %A, %B %e, %Y"), gatewayname, gatewayidnode)
  
    sc = SlackClient(_SLACK_TOKEN)
    response = sc.api_call("chat.postMessage", channel=_CHANNEL_NAME, blocks=blockmessage)
    print(blockmessage) 
    if not 'ok' in response or not response['ok']:
      print("Error posting message to Slack channel")
      print(blockmessage)
      print(response)
    else:
      print("Ok posting message to Slack channel")


report period 2019-03-21 15:37:49.033841 - 2019-03-21 16:52:49.033841
SELECT name, idnode, coreid, coretype_id, location_id FROM kanji_node WHERE meshnetwork_id=10000 AND deploystate_id=10001
Today is 2019-03-21 16:52:49.033841
The query returned the following:
xenon_04 was down 7 times in the period.
xenon_03 was up for the entire period.
xenon_01 was up for the entire period.
xenon_02 was down 1 times in the period.
argon_02 was up for the entire period.
There were a total of 8 downed nodes in the period.
We'll bounce the gateway (coreid=e00fce68c62dba103c893103) for this mesh network
https://api.particle.io/v1/devices/e00fce68c62dba103c893103/bounce
{'access_token': '790261410d6873b994fb6041553b5a99a7f7ed0e', 'arg': '_RESET_SECRET'}
The bounce request timed out as expected!
https://api.slack.com/img/blocks/bkb_template_images/notifications.png
[{'type': 'section', 'text': {'type': 'mrkdwn', 'text': '*Porter Farms - Building 1*     \nAt 4:52 PM Thursday, March 21, 2019     \n*Bounce-